In [1]:
import pandas as pd
from config import FieldConfig, ExperimentConfig
from pathlib import Path
DATA_PATH = Path('C:/Work/Data/proc/')
field_config = FieldConfig()

df = pd.read_csv(DATA_PATH/'rec_data2.csv')

missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

df_cleaned = df.dropna(subset=[field_config.date, field_config.amount, field_config.text, field_config.label])


Missing values per column:
id                    0
accountId             0
date                  0
amount                0
bankRawDescription    0
isRecurring           0
dtype: int64


In [2]:
from feature_processor import FeatProcParams
from config import *

feat_params = FeatProcParams()
feat_params_off = FeatProcParams(
    use_cyclical_dates=False,
    use_categorical_dates=False,
    use_continuous_amount=False,
    use_categorical_amount=False
)

exp_config = ExperimentConfig()

In [3]:
from runner import ExpRunner
from config import EmbModel
from embedder import EmbeddingService

runner1 = ExpRunner.create(
    exp_params=exp_config,
    full_df=df_cleaned,
    emb_params=EmbeddingService.Params(model_name=EmbModel.ALBERT),
    feat_proc_params=feat_params,
    field_config=FieldConfig()
)

C:\Users\yoav.haimovitch\PycharmProjects\PythonProject\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
runner1.run_torch([0.1])

Created 3 embedding layers. Total categorical embed dim: 112
Total input dim to MLP: 888


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'data.TrainingSample'>